Тут были эксперименты на разные модельки, которые нам не подошли.

При экспериментах с YOLOv8 и YOLOv10, стало очевидно, что эти модели значительно превосходят по метрикам и скорости работы такие системы, как RetinaNet и EfficientDet, что делает дальнейшую работу над ними практически бессмысленной, особенно с ограниченными квотами. Были предприняты попытки использовать PP-YOLOE, однако возникли проблемы с версиями и путями к базовым директориям, что затруднило реализацию. В результате, текущее состояние технологий указывает на необходимость сосредоточиться на более эффективных решениях, таких как предобученная в doclayout YOLOv10.


Хотя RetinaNet имеет потенциал для борьбы с дисбалансом классов (например, в документах часто преобладает класс "paragraph"), использование YOLOv10 позволяет добиться более стабильных результатов благодаря его оптимизированной архитектуре и способности к быстрой обработке. Поэтому, учитывая ограниченные ресурсы, целесообразнее сосредоточиться на использовании современных предобученных моделей для достижения лучших результатов.

Достаточно взглянуть на первые эпохи:   

RetinaNet (time $ \approx 80 $ мин) 
F1 Score: 0.3781
Mean IoU: 0.690

YOLOv8x  (time $ \approx 14 $ мин)
F1 Score: $ \approx 0.84 $
Mean IoU: $\approx 0.9 $
2

In [9]:
import os
import json
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import retinanet_resnet50_fpn
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from tqdm import tqdm

In [42]:
from torch.utils.data import Dataset
from PIL import Image, ImageFile
import torch
import json
import logging

# Обработка поврежденных изображений
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Настройка логирования
logging.basicConfig(level=logging.INFO)

class DocumentDataset(Dataset):
    def __init__(self, image_paths, annotation_paths, transform=None, target_transform=None):
        self.image_paths = sorted(image_paths)
        self.annotation_paths = sorted(annotation_paths)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.image_paths)

    def clip_boxes(self, boxes, image_width, image_height):
        """
        Обрезает координаты боксов, чтобы они не выходили за границы изображения.
        """
        boxes[:, 0] = torch.clamp(boxes[:, 0], min=0, max=image_width)
        boxes[:, 1] = torch.clamp(boxes[:, 1], min=0, max=image_height)
        boxes[:, 2] = torch.clamp(boxes[:, 2], min=0, max=image_width)
        boxes[:, 3] = torch.clamp(boxes[:, 3], min=0, max=image_height)
        return boxes

    def __getitem__(self, idx):
        try:
            # Загружаем изображение
            image = Image.open(self.image_paths[idx]).convert("RGB")
    
            # Загружаем аннотацию
            with open(self.annotation_paths[idx], 'r') as f:
                annotation = json.load(f)
    
            # Формируем bounding boxes
            boxes = torch.tensor(annotation.get("table", []) +
                                 annotation.get("title", []) +
                                 annotation.get("paragraph", []) +
                                 annotation.get("formula", []) +
                                 annotation.get("header", []) +
                                 annotation.get("footer", []) +
                                 annotation.get("footnote", []) +
                                 annotation.get("numbered_list", []) +
                                 annotation.get("marked_list", []) +
                                 annotation.get("table_signature", []) +
                                 annotation.get("picture_signature", []) +
                                 annotation.get("picture", []), dtype=torch.float32)
    
            # Формируем метки классов
            labels = torch.tensor([1] * len(annotation.get("table", [])) +
                                  [2] * len(annotation.get("title", [])) +
                                  [3] * len(annotation.get("paragraph", [])) +
                                  [4] * len(annotation.get("formula", [])) +
                                  [5] * len(annotation.get("header", [])) +
                                  [6] * len(annotation.get("footer", [])) +
                                  [7] * len(annotation.get("footnote", [])) +
                                  [8] * len(annotation.get("numbered_list", [])) +
                                  [9] * len(annotation.get("marked_list", [])) +
                                  [10] * len(annotation.get("table_signature", [])) +
                                  [11] * len(annotation.get("picture_signature", [])) +
                                  [12] * len(annotation.get("picture", [])), dtype=torch.int64)
    
            # Если аннотации пусты, создаем фиктивные значения
            if boxes.shape[0] == 0:
                boxes = torch.tensor([[0, 0, 0, 0]], dtype=torch.float32)
                labels = torch.tensor([0], dtype=torch.int64)
    
            # Проверяем границы боксов
            boxes = self.clip_boxes(boxes, annotation["image_width"], annotation["image_height"])
    
            # Удаляем боксы с нулевой шириной или высотой
            valid_indices = (boxes[:, 2] > boxes[:, 0]) & (boxes[:, 3] > boxes[:, 1])
            boxes = boxes[valid_indices]
            labels = labels[valid_indices]
    
            # Если после фильтрации боксов не осталось, создаем фиктивные значения
            if boxes.shape[0] == 0:
                boxes = torch.tensor([[0, 0, 0, 0]], dtype=torch.float32)
                labels = torch.tensor([0], dtype=torch.int64)
    
            # Создаем словарь целей
            targets = {"boxes": boxes, "labels": labels}
    
            # Преобразуем изображение
            if self.transform:
                image = self.transform(image)
    
            # Преобразуем цели, если нужно
            if self.target_transform:
                targets = self.target_transform(targets)
    
            return image, targets
    
        except (OSError, IOError, KeyError, json.JSONDecodeError) as e:
            # Логирование ошибок
            logging.warning(f"Пропущен файл или аннотация на индексе {idx} ({self.image_paths[idx]}): {e}")
            # Возвращаем фиктивное значение, чтобы не нарушить DataLoader
            dummy_image = torch.zeros(3, 224, 224)  # Замените на подходящий размер
            dummy_targets = {"boxes": torch.tensor([[0, 0, 0, 0]], dtype=torch.float32),
                             "labels": torch.tensor([0], dtype=torch.int64)}
            return dummy_image, dummy_targets


In [43]:
def collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        return None, None
    return tuple(zip(*batch))

In [44]:
images_dir = "/kaggle/input/doclayout-raw-data/raw_data/images"
annotations_dir = "/kaggle/input/doclayout-raw-data/raw_data/jsons"

image_paths = sorted([os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith('.png')])
annotation_paths = sorted([os.path.join(annotations_dir, f) for f in os.listdir(annotations_dir) if f.endswith('.json')])

# Разделение данных
train_images, val_images, train_annotations, val_annotations = train_test_split(
    image_paths, annotation_paths, test_size=0.1, random_state=42
)

transform = T.ToTensor()
train_dataset = DocumentDataset(train_images, train_annotations, transform=transform)
val_dataset = DocumentDataset(val_images, val_annotations, transform=transform)

In [55]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=4, pin_memory=True)

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = retinanet_resnet50_fpn(pretrained=True)
model.to(device)

optimizer = AdamW(model.parameters(), lr=0.001)

In [57]:
from tqdm import tqdm

def train_model(model, train_loader, val_loader, optimizer, device, num_epochs):
    """
    Обучение модели на нескольких эпохах.
    """
    model.to(device)
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        # Режим обучения
        model.train()
        train_loss = 0
        for images, targets in tqdm(train_loader, desc="Training"):
            if images is None or targets is None:
                continue  # Пропускаем пустые батчи
            
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            losses.backward()
            optimizer.step()
            train_loss += losses.item()
        
        train_losses.append(train_loss / len(train_loader))

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, targets in tqdm(val_loader, desc="Validation"):
                if images is None or targets is None:
                    continue  # Пропускаем пустые батчи

                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                val_loss += losses.item()

        val_losses.append(val_loss / len(val_loader))
        print(f"Epoch {epoch + 1}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

    return train_losses, val_losses


Эта ячейка была перезапущена, но ранее она обучалась около 80 минут:

In [ ]:
num_epochs = 1
train_losses, val_losses = train_model(model, train_loader, val_loader, optimizer, device, num_epochs)

Epoch 1/1


Training:  11%|█         | 481/4518 [06:07<51:05,  1.32it/s] 

In [1]:
from sklearn.metrics import precision_score, f1_score
from torchvision.ops import box_iou
import numpy as np

def evaluate_model(model, data_loader, device, iou_threshold=0.5):
    """
    Оценивает метрики модели: precision, F1 и mean IoU.

    Аргументы:
    - model: обученная модель.
    - data_loader: DataLoader для оценки (валидационный или тестовый набор).
    - device: устройство (CPU или GPU).
    - iou_threshold: порог IoU для определения True Positive.

    Возвращает:
    - mean_precision: средняя точность (precision).
    - mean_f1: средний F1-score.
    - mean_iou: средний IoU.
    """
    model.eval()
    all_precisions, all_f1s, all_ious = [], [], []

    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluating"):
            images = [img.to(device) for img in images]
            outputs = model(images)

            for i, output in enumerate(outputs):
                # Истинные рамки и классы
                true_boxes = targets[i]["boxes"].to(device)
                true_labels = targets[i]["labels"].to(device)

                # Предсказанные рамки и классы
                pred_boxes = output["boxes"]
                pred_labels = output["labels"]
                pred_scores = output["scores"]

                # Считаем IoU для всех предсказанных рамок с истинными
                ious = box_iou(pred_boxes, true_boxes)

                # Определяем True Positive, False Positive, False Negative
                true_positive = (ious.max(dim=1)[0] >= iou_threshold).sum().item()
                false_positive = len(pred_boxes) - true_positive
                false_negative = len(true_boxes) - true_positive

                # Precision, Recall, F1
                precision = true_positive / (true_positive + false_positive + 1e-6)
                recall = true_positive / (true_positive + false_negative + 1e-6)
                f1 = 2 * (precision * recall) / (precision + recall + 1e-6)

                # Средний IoU
                mean_iou = ious[ious >= iou_threshold].mean().item() if ious.numel() > 0 else 0.0

                # Добавляем метрики в список
                all_precisions.append(precision)
                all_f1s.append(f1)
                all_ious.append(mean_iou)

    # Возвращаем средние метрики
    return {
        "precision": np.mean(all_precisions),
        "f1": np.mean(all_f1s),
        "mean_iou": np.mean(all_ious),
    }

In [61]:
metrics = evaluate_model(model, val_loader, device)

print(f"Precision: {metrics['precision']:.4f}")
print(f"F1 Score: {metrics['f1']:.4f}")
print(f"Mean IoU: {metrics['mean_iou']:.4f}")

Evaluating: 100%|██████████| 502/502 [05:04<00:00,  1.65it/s]

Precision: 0.1944
F1 Score: 0.3781
Mean IoU: 0.6902
